# A Simple Hand-written Classifier
### ⭐인천대학교 임베디드시스템공학과 2023학년도 인공지능 교과목 과제 1번⭐


목표 : 자신이 만든 손글씨 이미지(t, u, v, w, x, y, z)를 학습하여 인식하는 모델 만들기  

Model 클래스를 구현하여 학습에 필요한 중요 변수들을 보관하였으며, 각 Model 클래스는  
Layer 클래스를 Property로 가져, 각 레이어에 필요한 정보들을 보관하였습니다.  
Layer의 객체는 Model 클래스 객체의 Property인 layers에 존재합니다.  
  
* 현재 설정된 모델  
Layer의 개수 : 4개  
각 Layer별 노드 개수 : 256(input), 96(hidden), 48(hidden), 7(output)  
=> 모든 레이어는 FC(Fully Connected Layer) Layer이다.  
  
코드의 처음 부분에서는 주어진 Path에서 Train(420개) Test(140개)를 Load하여 저장합니다.  
불러온 데이터는 Model Class의 객체에 저장됩니다.  
이 때 데이터를 가져오는 과정에서 모든 Pixel 값을 255로 나눠 Normalize합니다.  
또한 Label도 One hot encoding으로 만들어서 Model Class 객체에 함께 저장해 둡니다.
  
이후에는 하나의 Example씩 For loop을 돌면서 Forward Pass/Backward Pass를 진행합니다.  
=> 단, Parameter Update는 Batch Gradient Descent로 이루어진다. (Mini-batch는 사용하지 않음.)

Input은 16*16의 Image를 Flatten(256개의 값을 갖는 Vector)한 Vector이며,  
각 레이어 별로 다음 연산을 거칩니다. (Input Layer는 값을 넣어주기만 하는 역할)  
  
1) Z = WX + b (Input X에 가중치 Matrix W를 곱하고, bias b를 더해준다.)  
2) a = Activation(Z) (구해진 Z값을 activation function에 넣어 activation을 얻는다).  
=> hidden : ReLU 사용  
=> output : Softmax 사용  
  
이 과정을 거치면 최종적으로 (7, 1) size의 output이 나오게 되며, Softmax를 거쳤기 때문에  
각각의 element의 값이 0~1이고, 합이 1인 확률값으로 표현됩니다.  
이 값을 One hot encoding된 label과 비교하여 정답률을 계산합니다.    

본 코드는 Epoch, Learning Rate, Layer의 개수, Layer별 Node의 개수 등과 같은 Hyperparameter를 바꿔가면서  
학습을 진행해 볼 수 있도록 작성되었습니다. (해당 부분이 위치한 각 Cell별 설명에 작성하였음.)  
<br>

* 현재 설정한 모델의 구조  
    현재 모델의 레이어는 네 개이며, 각 레이어의 노드 개수는 [256, 96, 48, 7]입니다.  
    Epoch은 200, Learning Rate는 0.03이며, Training Data/Test Data는 420/140개 입니다.  
    이 설정을 바탕으로 한 개의 exmaple에 대한 forward pass/backward pass를 수행하고,  
    이 과정을 전체 example에 대해서 반복한 뒤에 Batch Gradient Descent로 parameter를 업데이트 합니다.  
    그리고 이 과정을 epoch 수만큼 반복한 뒤에, Test Data에 대한 inference 결과를 얻어냅니다.  

In [1]:
from PIL import Image
import math
import random
import os

* __Module Import__  (⬆️)

프로젝트에 필요한 Module을 Import 합니다.  
1) Image : png 이미지를 load하고 데이터 값을 읽어오는데 사용됩니다.  
2) math : 수학적 연산을 표현하는데 사용합니다.  
3) random : Parameter를 Initialization할 때 사용합니다.  
4) os : file을 읽을 때, 해당 파일의 경로를 표현하는데 사용합니다.

In [2]:
def get_target_vector(target_type):
    '''
        입력받은 데이터에 맞는 one hot encoded label을 만들어 반환하는 함수
    '''
    
    # file에서 얻어낸 target_value에 따라서
    # 매칭되는 target_vector를 return한다.
    if "t" in target_type:
        return [1, 0, 0, 0, 0, 0, 0]
    elif "u" in target_type:
        return [0, 1, 0, 0, 0, 0, 0]
    elif "v" in target_type:
        return [0, 0, 1, 0, 0, 0, 0]
    elif "w" in target_type:
        return [0, 0, 0, 1, 0, 0, 0]
    elif "x" in target_type:
        return [0, 0, 0, 0, 1, 0, 0]
    elif "y" in target_type:
        return [0, 0, 0, 0, 0, 1, 0]
    elif "z" in target_type:
        return [0, 0, 0, 0, 0, 0, 1]

* __get_target_vector()__ (⬆️)

Train/Test 데이터를 load할 때, 읽어낸 데이터 파일의 이름에서 target label의 type을 분리합니다.  
이 값을 get_target_vector() 함수에 입력하면 one hot encoding된 target label vector를 반환합니다.  
(t, u, v, w, x, y, z) 7개의 label type이 존재하므로, 7개의 type의 vector를 반환합니다.  

In [3]:
def parse_img_pixels(path, data_type):
    '''
        주어진 path에 들어있는 학습 데이터를 읽고,
        RGB pixel value를 저장하는 함수
    '''
    
    loaded_data = []
    target = []
    
    for file_name in os.listdir(path):
        img = Image.open(path+file_name)
        img_data = img.getdata()
        
        target_value = file_name.split('_')
        
        if (data_type == "train"):
            # target_value 저장
            target_value = target_value[0]
        elif(data_type == "test"):
            target_value = target_value[1]
        
        # target의 type(t, u, v, w, x, y, z)에 따른 label을 반환하고, 
        # 이를 data set의 전체 target을 보관하는 list에 저장한다.
        target_vector = get_target_vector(target_value)
        target.append(target_vector)
        
        pixel_values = [] # 하나의 img를 구성하는 pixel 값을 보관하는 list
        img_len = len(img_data) # img example의 길이(16*16 = 256)
        
        # 하나의 이미지에 들어있는 모든 픽셀에 대해서 loop 동작
        for pixel_index in range(img_len):
            
            # 각 pixel 값을 255로 나눠서 float로 변환해 표현력을 증가시키고,
            # 값의 범위를 0~1 사이로 줄여서 forward pass 과정에서 곱해지는
            # 값의 크기에 의한 영향을 줄인다. (값의 절댓값이 클수록 activation이 커지는 현상을 감소)
            pixel_values.append(img_data[pixel_index]/255)
        
        # 하나의 example의 모든 pixel 값을 보관한 pixel_values를 loaded_data에 추가한다.
        # 따라서 모든 for loop을 돌면, loaded_data는 train or test의 모든 data set을 가지게 될 것이다.
        loaded_data.append(pixel_values) 
    
    return loaded_data, target

* __parse_image_pixels()__ (⬆️)

인자로 전달되는 path(image의 경로 + 이름)와 train/test 여부에 따라 이미지 데이터로부터 값을 읽는 함수입니다.  
읽어낸 이미지에서 data와 label을 분리하고, data를 255로 나눠줍니다. (Normalization)  
  
data를 255로 나누는 것은 pixel 값의 범위가 0~255이기 때문인데, 간단하게 255로 값을 나눠줌으로써  
float로 변환하여 표현력을 증가시키고, 값의 절댓값에 의해 activation이 커지는 문제를 완화합니다.  

In [4]:
def relu(input):
    return max(input, 0)

def element_wise_relu(input):
    '''
        hidden layer에 사용 될 activation function(ReLU)
        input vector를 전달받아서 각각에 대해 relu를 적용한 뒤 결과를 반환한다.
    '''
    
    activation_len = len(input)
    
    for activation_idx in range(activation_len):
        input[activation_idx] = relu(input[activation_idx])
    
    return input

def softmax(input):
    '''
        output layer에 사용 될 activation function(Softmax)
        t, u, v, w, x, y, z 각각에 대해서 정답일 확률을 출력한다.
    '''
    
    total_exp_value_sum = 0
    activation = []
    
    max_value = max(input)
    
    for idx in range(len(input)):
        exp_value = math.exp(input[idx] - max_value)
        total_exp_value_sum += exp_value
        activation.append(exp_value)
    
    for idx in range(len(input)):
        activation_value = activation[idx] / total_exp_value_sum
        activation[idx] = activation_value
    
    return activation

* __activation functions__ (⬆️)

Model의 Layer를 구성하는데 사용되는 Acitvation Function을 구현한 Cell입니다.  
Hidden Layer에서는 ReLU를 사용하고, Output Layer에서는 Softmax를 사용합니다.  
1) ReLU : input이 0보다 작거나 같으면 0을 출력, 양수라면 값을 그대로 출력합니다.
2) Softmax : Multiple Classification에 사용되며, 현재 모델에서는 (7,1)의 결과를 출력합니다.  
각 값이 0 ~ 1 사이의 확률값으로 표현되며, 전체 합은 1이기 때문에 결과적으로 (t, u, v, w, x, y, z) 중  
하나의 값을 선택하는 역할을 합니다.

In [5]:
def dot_product(prev_node_num, cur_node_num, weight, data):
    '''
        Forward Pass 과정에서, 각 Layer에서 행렬곱 함수
    '''

    output = []
    
    for i in range(cur_node_num):
        value = 0
        for j in range(prev_node_num):
            value += weight[i][j] * data[j]
        output.append(value)
        
    return output

* __dot_product()__ (⬆️)

forward pass 과정에서 Z = WX + b 연산을 진행하는데 필요한 행렬곱 연산을 수행하는 함수입니다.  
이전 노드의 개수와 현재 노드의 개수를 고려하면 weight와 bias, 그리고 data(Input)의 길이를 알 수 있으므로,  
이 값들을 전달받아 이중 for loop을 통해 행렬곱을 진행합니다.  

In [6]:
def init_params_by_type(cur_layer_node_num, prev_layer_node_num, init_type):
    ''' 
        Layer의 weight를 초기화하는 함수
        init_type에 따라 Xavier 또는 He Initialization을 사용한다.
    '''
    
    weight = []
    bias = []
    
    dW = []
    db = []
    
    std = 0
    
    if (init_type == "xavier"):
        std = math.sqrt(1.0 / prev_layer_node_num)  
    elif (init_type == "he"):
        std = math.sqrt(2.0 / prev_layer_node_num)
    
    for cur_node_idx in range(cur_layer_node_num):
        # bias는 모두 0으로 초기화한다.
        # 또한, 학습에 사용할 gradient를 저장할 db는 bias와 똑같은 크기이므로,
        # 이와 같은 크기로 모든 원소를 0을 가지도록 초기화한다.
        bias.append(0)
        db.append(0)
        
        # weight는 Xavier Initialization을 사용.
        # gradient를 저장할 dW도 weight와 똑같은 크기로
        # 모든 원소를 0을 가지도록 초기화한다.
        weight_row = []
        dW_weight_row = []
        
        for prev_node_idx in range(prev_layer_node_num):
            # 평균은 0이고 표준편차는 std인 분포에서 random값을 선택한다.
            random_number = random.gauss(0, std) 
            
            weight_row.append(random_number) # 선택한 random값을 weight_row에 추가
            dW_weight_row.append(0) # dW_weight_row는 gradient를 보관하므로, 처음에는 0으로 초기화
            
        weight.append(weight_row) # weight_row를 weight에 추가
        dW.append(dW_weight_row) # dW_weight_row를 dW에 추가
    
    return weight, bias, dW, db

* __init_params_by_type()__  
  
현재 Layer와 이전 Layer의 Node 개수를 전달받고,이 크기를 바탕으로  
Weight, bias를 Random Initialize 합니다.
  
이 때, ReLU를 사용하는 Layer(Hidden Layer)는 He Initialization을 사용하고,  
Softmax를 사용하는 Layer(Output Layer)는 Xavier Initialization을 사용합니다.

(Xavier Initialization의 경우 실제로는 LeCun이지만 참고한 교재에서  
Xavier Initialization이라고 지칭하여, 현재 프로젝트에서도  
Xavier Initialization이라고 지정해 두었습니다.)
  
또, 각 Layer에서 Backward pass에서 gradient를 저장할 리스트가 필요하기 때문에,  
이 과정에서 Weight와 bias의 크기와 똑같이 dW, db를 모두 0으로 초기화합니다.

# 모델을 구성하는 전체 Class 선언
  
1) *__Layer Class__*  
Model을 구성하는 __Layer Class__ 선언  
  
2) *__Model Class__*  
Layer 객체를 갖는 __Model Class__ 선언  

In [7]:
class Layer:
    '''
        Fully Connected Layer Class
    '''

    def __init__(self, activation_type, layer_idx):
        '''
            Layer 객체의 생성자
        '''
        
        # 이 Layer가 갖는 activation의 type
        # C언어에서는 enum으로 지정한다.
        self.activation = activation_type
        
        # 이 Layer가 몇 번째 Layer인지 나타내는 index 변수
        # input layer를 포함한다.
        self.layer_idx = layer_idx
        
        # weight
        # 현재 Layer가 갖는 가중치값이다.
        # Size : (prev_layer_node_num, current_layer_node_num)
        self.weight = []
        
        # bias
        # 현재 Layer가 갖는 편향값이다.
        # Size : (current_layer_node_num, 1)
        self.bias = []
        
        # backward pass 과정에 필요한 값을 저장하는 cache list
        # weight와 bias는 Layer 자체에서 계속해서 저장하므로,
        # prev_activation과 Z값만 저장하면 된다.
        self.cache = {}
        
        self.dW = []
        self.db = []

    def initialize_parameter(self, cur_layer_node_num, prev_layer_node_num, layer_num):
        '''
            현재 Layer의 Parmeter(Weight, Bias)값을 초기화하는 메서드
        '''
        
        # Layer를 구성하는 parameter를 initialize 한다.
        
        # 1. bias
        # bias는 size가 (cur_layer_node_num, 1)이므로, 모두 0으로 초기화한다.
        
        # 2. weight
        # weight는 size가 (cur_layer_node_num, prev_layer_node_num)이다.
        
        # output layer라면 activation으로 Softmax를 사용하므로, Xaiver Initialization을 사용한다.
        if (self.layer_idx == layer_num-1):
            self.weight, self.bias, self.dW, self.db = init_params_by_type(cur_layer_node_num, prev_layer_node_num, "xavier")
        # hidden layer라면 activation으로 ReLU를 사용하므로, He Initialization을 사용한다.
        else:
            self.weight, self.bias, self.dW, self.db = init_params_by_type(cur_layer_node_num, prev_layer_node_num, "he")
    
    def save_cache_variable(self, prev_activation, Z):
        '''
            forward pass에서, 각 Layer의 학습에 필요한
            cache 값을 저장하는 메서드
        '''
        
        self.cache['prev_activation'] = prev_activation
        self.cache['Z'] = Z
        
    def save_gradient(self, dW, db, cur_layer_node_num, prev_layer_node_num):
        '''
            backward pass에서, 한 개의 example에 대해서 구해진 gradient를
            현재 Layer의 dW, db에 더해주는 메서드. 

            parameter update는 전체 example에 대한 총합을 구하는 과정이며, 
            각 example에 대한 값으 모두 구한 뒤에 update_parameter() 메서드를 통해 
            example 개수로 나눠서 Batch Gradient Descent를 적용한다.
        '''
        
        for i in range(cur_layer_node_num):
            self.db[i] += db[i]
            for j in range(prev_layer_node_num):
                self.dW[i][j] += dW[i][j]
    
    def update_parameter(self, train_data_num, cur_layer_node_num, prev_layer_node_num, learning_rate):
        '''
            구해진 전체 dW, db를 exaple의 개수로 나눠서 1 epoch에 대한 gradient를 구하는 메서드
        '''
        
        for i in range(cur_layer_node_num):
            self.db[i] /= train_data_num
            for j in range(prev_layer_node_num):
                self.dW[i][j] /= train_data_num
        
        for i in range(cur_layer_node_num):
            self.bias[i] -= learning_rate * self.db[i]
            for j in range(prev_layer_node_num):
                self.weight[i][j] -= learning_rate * self.dW[i][j]
        
    def clear_gradient(self, cur_layer_node_num, prev_layer_node_num):
        '''
            1 epoch에 대해서 학습을 진행한 뒤, dW, db 값을 0으로 초기화하는 메서드
            이를 통해 다음 epoch에서의 gradient를 구해서 update를 진행할 수 있도록 한다.
        '''
        
        for i in range(cur_layer_node_num):
            self.db[i] = 0
            for j in range(prev_layer_node_num):
                self.dW[i][j] = 0
        
        

* *__Layer Class__*

<br>

1) __init()__  
  
    Layer Class의 생성자입니다.  
    Layer는 다음과 같은 변수를 가집니다.  

    
     
    * activation : 해당 Layer의 Activation Function을 결정합니다. (ReLU, Softmax)  

    * layer_idx : 해당 Layer의 Index를 결정합니다. (Input Layer는 0)  

    * weight : 해당 Layer의 Weight를 갖는 리스트입니다.  

    * bias : 해당 Layer의 bias를 갖는 리스트입니다.  

    * cache : Backward Pass에 사용할 Z값(레이어 별 행렬곱의 결과), previous Activation(현재 Layer의 Input)을 보관하는 객체입니다.  

    * dW : Weight의 gradient를 보관하는 리스트입니다.  

    * db : bias의 gradient를 보관하는 리스트입니다.  

<br>

2) __initialize_parameter()__  

    Parameter를 초기화하는 함수입니다. 인자로 현재 레이어, 이전 레이어의 노드 개수를 전달받고,  
    전체 레이어의 개수를 전달받습니다.  
    
    hidden layer는 ReLU를 사용하고, output layer는 Softmax를 사용합니다.  
    각각의 type에 따라서 init_params_by_type() 함수를 사용해 Weight/bias를 초기화합니다.

<br>

3) __save_cache_variable()__  

    Backward Pass에서 Gradient를 만드는데 필요한 값을 Forward Pass 과정에서 저장하는 함수입니다.

<br>

4) __save_gradient()__  

    Backward Pass 과정에서 구한 gradient를 dW와 db에 저장하는 함수입니다.

<br>

5) __update_parameter()__  

    Backward Pass에서 구한 gradient를 이용해 parameter를 update하는 함수입니다.

<br>

6) __clear_gradient()__  

    Backward Pass에서 구한 gradient를 삭제하고, dW, db를 모두 0으로 초기화하는 함수입니다.  
    한 번의 Backward Pass가 실행이 완료되면, 다음 Example에 대한 gradient를 구해야 하기 때문에 실행합니다.

<br>


In [8]:

class Model:
    '''
        FC Layer를 property로 갖는 Model Class
    '''
    
    
    def __init__(self, layer_num, epoch_num, learning_rate):
        '''
            Model 객체의 생성자
        '''
        
        # input layer를 포함한 전체 layer의 개수
        self.layer_num = layer_num
        
        # input layer를 포함하여, Layer 객체를 보관하는 리스트
        self.layers = [] 
        
        # 각 layer 별 node의 개수를 보관하는 리스트
        # 추론/학습 과정에서 input layer의 노드 개수가 필요하므로, 노드 개수도 보관한다.
        self.layer_node_num = []
        
        # train/test data
        # 16*16 이미지가 하나의 example이므로, 하나의 example의 size는 (256, 1)
        # 전체 데이터 셋이 m개일 때, 전체 data set의 size는 (256, m)이라고 할 수 있다.
        self.train_data_num = 0 # 420개 (t, u, v, w, x, y, z 60세트)
        self.train_data = [] # 420개의 train data를 보관하는 list
        
        self.test_data_num = 0 # 140개 (t, u, v, w, x, y, z 20세트)
        self.test_data = [] # 140개의 test data를 보관하는 list
        
        # data target
        # train/test에 대한 target이다.
        # 하나의 target은 (1, 7) size의 list이며,
        # train은 420개이므로, size는 (420, 7)
        # test는 140개이므로, size는 (140, 7)
        self.train_target = []
        self.test_target = []
        
        # 전체 데이터셋을 학습할 횟수, epoch num
        self.epoch_num = epoch_num
        
        # 학습률(learning rate)
        self.learning_rate = learning_rate
    
    def load_data(self, train_data_path, test_data_path):
        '''
            지정된 경로에서 train/test data set를 불러오는 메서드
        '''
        
        # train/test data를 정해진 경로에서부터 load하여 저장
        self.train_data, self.train_target = parse_img_pixels(train_data_path, "train")
        self.test_data, self.test_target = parse_img_pixels(test_data_path, "test")
        
        # train/test 데이터의 개수를 저장
        self.train_data_num = len(self.train_data)
        self.test_data_num = len(self.test_data)
    
    
    def insert_new_layer(self, activation_type, layer_idx):
        '''
            Model에 새로운 Layer를 추가하는 메서드
            
            insert_new_layer는 반드시 input layer가 만들어진 이후에 실행되므로,
            hidden layer부터 사용되기 때문에 layer_num의 indexing에는 문제 없음.
        '''
        
        # layer_idx는 input layer를 포함했을 때, 해당 layer의 index이다.
        # 예를 들어, 첫 번째 hidden layer의 index는 1이다. (input layer는 0)
        layer = Layer(activation_type, layer_idx) # Layer 객체 생성
        
        if (activation_type != 'none'): # input layer에 대해서는 실행하지 않음.
            cur_layer_idx = layer_idx
            prev_layer_idx = cur_layer_idx-1
            
            # 현재 Layer가 갖는 Weight/bias를 초기화한다.
            layer.initialize_parameter(cur_layer_node_num=self.layer_node_num[cur_layer_idx], 
                                    prev_layer_node_num=self.layer_node_num[prev_layer_idx],
                                    layer_num=self.layer_num)
        
        # Layer 객체를 self.layers에 추가한다.
        self.layers.append(layer)
            
    def forward_pass(self, data_index, data_type):
        '''
            추론(inference) 과정을 진행하는 메서드
            hidden layer에서는 ReLU를 거쳐 activation을 내보내고,
            output layer에서는 Softmax를 거쳐 t, u, v, w, x, y, z에 대한 확률값을 반환한다.
        '''
        
        # activation이라는 변수는 다음 layer에 들어가는 input을 의미하며,
        # for loop을 돌면서, 한 layer에서 반환된 activation을 다음 layer로 넘겨주는 역할이다.
        # 구현 상의 편의를 위해 hidden layer 1에 들어가는 input data를 activation에 초기화한다.
        # 즉, 이전 layer의 activation이 다음 layer에 대해서는 input이 되며, 이를 하나의 변수로 처리하는 것이다.
        activation = 0
        
        # train/test에 따라서 처음에 선택되는 input값을 선택한다.
        if (data_type == "train"):
            activation = self.train_data[data_index]
        elif (data_type == "test"):
            activation = self.test_data[data_index]
        
        # forward pass, inference를 진행한다.
        # 예를 들어, layer_num = 4일 경우 (input, hidden, hidden, output) 세 번의 loop가 동작하게 될 것이다.
        for layer_index in range(1, self.layer_num):
            cur_layer_idx = layer_index
            prev_layer_idx = cur_layer_idx-1
            
            current_layer_node_num = self.layer_node_num[cur_layer_idx] # 현재 layer의 node 개수
            prev_layer_node_num = self.layer_node_num[prev_layer_idx] # 이전 layer의 node 개수
            
            # prev_activation은 이전 layer에서 현재 layer로 전달되는 activation 값이다.
            # 초기에는 input data가 activation임에 유의한다.
            prev_activation = activation
            
            # 행렬곱을 진행하는 부분이다. 
            # 데이터에 weight를 곱하고 bias를 더한다.
            # 그리고 이 연산의 결과를 Z라고 한다.
            Z = dot_product(
                prev_node_num=prev_layer_node_num, 
                cur_node_num=current_layer_node_num, 
                weight=self.layers[cur_layer_idx].weight, 
                data=activation
            )
            
            # 출력된 z vector에 대해서 bias를 각각 더해준다.
            for i in range(self.layer_node_num[cur_layer_idx]):
                Z[i] += self.layers[cur_layer_idx].bias[i]
                
            # backward_pass에 필요한 값들을 cache에 저장해둔다.
            # weight와 bias는 layer 객체에 저장되어 있으므로, prev_activation과 z만 보관한다.
            self.layers[cur_layer_idx].save_cache_variable(prev_activation, Z)
            
            # z를 활성화 함수에 넣어서 activation 값을 얻는다.
            if (self.layers[cur_layer_idx].activation == "relu"):
                activation = element_wise_relu(Z)
            elif (self.layers[cur_layer_idx].activation == "softmax"):
                activation = softmax(Z)
        
        # forward pass의 최종 출력(output)을 return한다.
        return activation

    def calc_output_layer_dZ(self, output_layer_node_num, output, cur_target):
        '''
            Softmax(Output Layer Activation Function) + Cross Entropy(Loss Function)를 사용하면
            미분 식이 매우 간단해져서 gradient를 쉽게 구할 수 있다.
            이 메서드에서는 아래 for loop 계산을 통해서, output layer에 대한 dZ를 구해 return 한다.
        '''
        dZ = []
        
        for idx in range(output_layer_node_num):
            dZ.append(output[idx] - cur_target[idx])
            
        return dZ
    
    def calc_hidden_layer_dZ(self, dA, cur_layer_idx, cur_layer_node_num):
        '''
            hidden layer에서의 dZ를 계산하는 함수
            ReLU 공식에 따라, Forward 과정에서 Z값이 0보다 작거나 같았다면
            Gradient는 0이고, Z값이 0보다 크다면 Gradient는 1이다.
            이에 따라 Chain Rule을 사용하여 모든 Layer에 대한 dZ Gradient를 계산한다.
        '''

        # forward pass에서 계산했던 Z값을 cache로부터 가져온다.
        Z = self.layers[cur_layer_idx].cache['Z']
        
        dZ_value = [] # dA/dZ를 저장하는 리스트
        
        # ReLU의 Gradient는 Z값이 0보다 클 경우 1, 0보다 작을 경우 0으로 처리하므로
        # 다음과 같이 Z를 탐색하며 0보다 클 경우 dZ_value에 1을 추가, 작다면 0을 추가한다.
        for i in range(cur_layer_node_num):
            if (Z[i] < 0):
                dZ_value.append(0)
            else:
                dZ_value.append(1)
    
        dZ = []
        
        # dZ를 계산한다. dZ는 dA*dZ_value이다.
        # dA : 앞의 Layer에서 넘어온 미분값
        # dZ_value : 위에서 구한, ReLU에 대한 미분값
        # 둘을 곱하면 완전한 dZ를 얻게 된다.
        for i in range(cur_layer_node_num):
            dZ.append(dA[i]*dZ_value[i])
        
        return dZ

    def calc_dW(self, cur_layer_node_num, prev_layer_node_num, cur_layer_idx, dZ):
        '''
            Output Layer의 Weight Gradient를 계산하는 함수
        '''
        
        # forward pass 과정에서 저장해 두었던 prev_activation을 cache에서 불러온다.
        prev_activation = self.layers[cur_layer_idx].cache['prev_activation']
        dW = []
        
        # Output Layer의 Weight Gradient 계산
        for i in range(cur_layer_node_num): 
            weight_row = []
            for j in range(prev_layer_node_num):
               weight_value = dZ[i] * prev_activation[j]
               weight_row.append(weight_value)
            dW.append(weight_row)
        
        return dW
    
    def calc_db(self, dZ):
        '''
            db를 계산하는 메서드
            현재 사용하는 모델에서 db의 gradient는 dZ와 동일하므로,
            dZ를 그대로 return해준다. => b로 편미분하면 db = dZ이다.
            (추후에 변경될 가능성을 고려하여 모듈화함)
        '''
        
        return dZ
        
    def calc_dA_prev(self, cur_layer_node_num, prev_layer_node_num, cur_layer_idx, dZ):
        '''
            현재 레이어의 prev_activation에 대한 gradient인 dA_prev를 계산하는 메서드
            이 값을 계산하여 이전 레이어의 gradient를 계산하는데 전달해 주어야 한다.
            (Hidden Layer는 직접적인 Error를 구할 수 없으며, Output Layer에서 Gradient를 계산하며
            점차 이전 노드로 전파해 주는 방식을 사용해야 한다.)
        '''

        dA_prev = []
        
        for i in range(prev_layer_node_num):
            dA_prev_value = 0
            
            for j in range(cur_layer_node_num):
                dA_prev_value += self.layers[cur_layer_idx].weight[j][i] * dZ[j]
            dA_prev.append(dA_prev_value)
        
        return dA_prev

    def calc_softmax_gradient(self, cur_layer_idx, cur_layer_node_num, output, train_data_index):
        '''
            Output Layer의 Gradient를 계산하는 메서드
            dW, db를 계산하기 위해서는 dA_prev, dZ가 필요하다.
            
            그런데 현재 Output Layer가 Softmax를 사용하기 때문에
            dZ를 간단하게 축약하여 구할 수 있으며(calc_output_layer_dZ), 
            이 값과 weight, bias의 편미분을 곱해서 계산하면 gradient를 구할 수 있다.
            (calc_dW, calc_db)
            
            또한 Output Layer에서는 dA_prev를 계산하여 이전 레이어로 전파해 주어야 하기 때문에
            calc_dA_prev를 실행하여 값을 구하고 return 한다.
        '''

        # Gradient 계산 과정에서 Output 값이 필요하므로 target 값을 가져온다.
        # 현재 학습을 진행 중인 example의 index는 train_data_index이다.
        cur_target = self.train_target[train_data_index] 

        # 행렬곱 연산을 수행하기 위해서 이전 layer의 index와 node_num을 가져온다.
        prev_layer_idx = cur_layer_idx-1
        prev_layer_node_num = self.layer_node_num[prev_layer_idx]
    
        # dW와 db를 계산한다.
        # 1) dW, db를 계산하기 위해서는 dZ가 필요하므로, 먼저 dZ를 계산한다.
        # 2) dZ와 prev_activation을 바탕으로 dW를 계산한다.
        # 3) dZ를 바탕으로 db를 계산한다.
        dZ = self.calc_output_layer_dZ(cur_layer_node_num, output, cur_target)
        dW = self.calc_dW(cur_layer_node_num, prev_layer_node_num, cur_layer_idx, dZ)
        db = self.calc_db(dZ)
        
        # dZ와 Weight를 이용해, 이전 layer로 전달해 줄 dA_prev(prev_activation의 gradient)를 계산한다.
        dA_prev = self.calc_dA_prev(cur_layer_node_num, prev_layer_node_num, cur_layer_idx, dZ)
        
        # 구한 dW, db, dA_prev를 반환한다.
        return dW, db, dA_prev
       
    # 인자로 넘어오는 layer_idx는 input layer를 포함한 index임에 유의한다.
    def calc_relu_gradient(self, dA, cur_layer_node_num, cur_layer_idx):
        '''
            Hidden Layer의 Gradient를 계산하는 메서드
            
            dA : 현재 노드의 Activation에 대한 Gradient이다. 이 값은
            next_layer에서 연산하여 현재 레이어로 전달해 주는 값이다.

            layer_idx : 현재 Layer의 index이다.
        '''
        
        # 행렬곱 연산을 수행하기 위해서 이전 layer의 index와 node_num을 가져온다.
        prev_layer_idx = cur_layer_idx-1
        prev_layer_node_num = self.layer_node_num[prev_layer_idx]
        
        # dW와 db를 계산한다.
        # 1) dW, db를 계산하기 위해서는 dZ가 필요하므로, 먼저 dZ를 계산한다.
        # 2) dZ와 prev_activation을 바탕으로 dW를 계산한다.
        # 3) dZ를 바탕으로 db를 계산한다.
        dZ = self.calc_hidden_layer_dZ(dA, cur_layer_idx, cur_layer_node_num)
        dW = self.calc_dW(cur_layer_node_num, prev_layer_node_num, cur_layer_idx, dZ)
        db = self.calc_db(dZ)
        
        # dZ와 Weight를 이용해, 이전 layer로 전달해 줄 dA_prev(prev_activation의 gradient)를 계산한다.
        dA_prev = self.calc_dA_prev(cur_layer_node_num, prev_layer_node_num, cur_layer_idx, dZ)
        
         # 구한 dW, db, dA_prev를 반환한다.
        return dW, db, dA_prev
    
    def backward_pass(self, output, train_data_index): 
        '''
            학습 과정(Backward Pass)이자, parameter update 과정이다.
            Gradient Descent에 의해 Error를 바탕으로 학습을 진행한다.
        '''
        
        # Output Layer에 대한 gradient 계산 진행 (Softmax)
        output_layer_idx = self.layer_num-1 # output_layer_idx == self.layer_num-1
        output_layer_node_num = self.layer_node_num[output_layer_idx]
        
        prev_layer_idx = output_layer_idx-1 # the prev layer of output layer
        prev_layer_node_num = self.layer_node_num[prev_layer_idx]
        
        # output layer에 대한 gradient를 계산한다. (Softmax)
        dW, db, dA_prev = self.calc_softmax_gradient(output_layer_idx, output_layer_node_num, output, train_data_index)
        
        # 전달받은 dW와 db를 output layer에 저장한다.
        self.layers[output_layer_idx].save_gradient(dW, db, output_layer_node_num, prev_layer_node_num)
        
        # hidden layer에 대한 gradient를 계산한다. (ReLU)
        # layer_idx는 input layer를 포함한 index.
        for layer_idx in range(self.layer_num-2, 0, -1):
            cur_layer_idx = layer_idx
            prev_layer_idx = cur_layer_idx-1
            
            cur_layer_node_num = self.layer_node_num[cur_layer_idx]
            prev_layer_node_num = self.layer_node_num[prev_layer_idx]
            
            dW, db, dA_prev = self.calc_relu_gradient(dA_prev, cur_layer_node_num, cur_layer_idx)
            
            self.layers[cur_layer_idx].save_gradient(dW, db, cur_layer_node_num, prev_layer_node_num)
            
    def isCorrect(self, output, train_data_index):
        '''
            inference의 결과가 정답인지 확인하는 메서드
        '''

        # 1 epoch 내에서 1 example에 대한 결과의 정답 여부를 체크
        predict_index = output.index(max(output))
        target_index = self.train_target[train_data_index].index(1)
        
        # 정답일 경우 correct를 1 증가시킨다.
        if predict_index == target_index:
            return True
        else:
            return False
        
    def train(self):
        '''
            training 과정을 진행하는 메서드
            training data에 대한 inference/learning을 수행한다.
        '''

        # 정해진 epoch num만큼 학습을 진행
        total = self.train_data_num # train data set의 개수
        
        for epoch in range(self.epoch_num):
            # 1 epoch 학습을 진행
            print(f"Epoch {epoch}")
        
            correct = 0
            for train_data_index in range(self.train_data_num):
                # 1개의 example에 대한 forward_pass 결과를 얻는다.
                output = self.forward_pass(train_data_index, "train")
                
                # 정답일 경우 correct를 1 증가시킨다.
                if self.isCorrect(output, train_data_index):
                    correct += 1
                
                # Loss를 backward_pass에 넣어서 1 example에 대한 gradient를 계산한다.
                self.backward_pass(output, train_data_index)
                
            # 1 epoch에 대한 Accuracy를 출력
            print (f"Accuracy : {correct/total}")            
            
            # 1 epoch에 대한 가중치 업데이트 진행(Batch Gradient Descent)
            for layer_idx in range(1, self.layer_num):
                cur_layer_idx = layer_idx
                prev_layer_idx = cur_layer_idx-1
                
                cur_layer_node_num = self.layer_node_num[cur_layer_idx]
                prev_layer_node_num = self.layer_node_num[prev_layer_idx]
                
                self.layers[layer_idx].update_parameter(self.train_data_num, cur_layer_node_num, 
                                                        prev_layer_node_num, self.learning_rate)    
                
                self.layers[layer_idx].clear_gradient(cur_layer_node_num, prev_layer_node_num) 
    
    def predict(self):
        '''
            학습된 Parameter 값을 가지고 test set에 대한 forward pass를 진행한다.
        '''
        
        correct = 0
        wrong = 0
        
        for test_data_idx in range(self.test_data_num):
            output = self.forward_pass(test_data_idx, "test")
            predict_index = output.index(max(output))
            target_index = self.test_target[test_data_idx].index(1)
            
            if predict_index == target_index:
                correct += 1
            else:
                wrong += 1
        print(f"Correct : {correct}")
        print(f"Total : {self.test_data_num}")
        print(f"정답률 : {correct/self.test_data_num}")
            

* *__Model Class__*

<br>

1) __init()__  

    Layer Class의 생성자입니다.  
    Layer는 다음과 같은 Property를 가집니다.  

    
     
    * layer_num : Model을 구성하는 전체 Layer의 개수입니다. Input Layer를 포함합니다.

    * layers : Model을 구성하는 Layer의 객체를 보관하는 리스트입니다.

    * layer_node_num : Layer 별 Node의 개수를 보관하는 리스트입니다.

    * train_data_num : Training Data의 전체 개수입니다.

    * train_data : Training Data를 보관하는 리스트입니다.

    * test_data_num : Test Data의 전체 개수입니다.

    * test_data : Test Data를 보관하는 리스트입니다.

    * train_target : Training Data에 대한 Target Label Vector를 보관하는 리스트입니다.

    * test_target : Test Data에 대한 Target Label Vector를 보관하는 리스트입니다.

    * epoch_num : 전체 Epoch 횟수입니다.

    * learning_rate : 학습 과정에서 적용할 학습률입니다.

<br>

2) __load_data()__

    Training/Test Data를 load하는 함수입니다.  
    parse_img_pixels() 함수를 이용하여 각각의 path에서 데이터를 load하고,  
    data/target을 구분하여 반환받아 클래스에 저장합니다.  
    그리고, 각 Training/Test의 개수를 data_num에 저장합니다.
  
<br>

3) __insert_new_layer()__

    Model을 구성하는 새로운 Layer를 만들고, Layer의 객체를 self.layers에 저장하는 함수입니다.  
    인자로 activation type을 전달받아 Layer에 전달해주며, Input Layer는 layer_node_num과  
    layers의 index가 매칭되도록 만들기 위한 용도이므로, Layer 객체를 만들어서  
    self.layers에 추가하지만, 가중치를 초기화 하지는 않습니다.  

<br>

4) __forward_pass()__ 
  
    Forward Pass를 진행하는 함수입니다.  
    Train/Test 과정에서 공통적으로 사용하기 위해 data_type(train or test)를 전달받아 구분합니다.  
    처음에 input 데이터를 activation 변수에 넣고, 각 Layer를 거치면서 생성되는  
    Layer별 새로운 activation값으로 계속해서 갱신합니다.  
    Input Layer는 가중치를 가지지 않으므로, hidden Layer부터 계산을 하도록  
    for loop를 돌리며, 안에서는 다음 두 개의 연산을 진행합니다.  
      
    1) Z = WX + b (Input X에 가중치 Matrix W를 곱하고, bias b를 더해준다.)  
    2) a = Activation(Z) (구해진 Z값을 activation function에 넣어 activation을 얻는다).  
      
    이렇게 최종적으로 얻어낸 output(activation 변수에 저장되어 있음)을 return합니다.
    
<br>

5) __calc_output_layer_dZ()__  

    Backward Pass 과정에서 output layer(Softmax Layer)에서의 dZ를 계산하는 함수입니다.  
    본 프로젝트에서 dA, dZ의 의미는, 다음과 같습니다.  
    * dA : Loss를 해당 layer의 activation으로 편미분한 값.  
    * dZ : Loss를 해당 layer의 Z로 편미분한 값.  
      
    Backward Pass에서 parameter의 gradient를 구하기 위해 dZ가 필요하므로, 이 함수에서 계산합니다.  
    그런데, output layer는 Softmax를 Activation Function으로 사용합니다.  
    현재 Loss Function을 Cross Entropy로 사용하고 있으므로, 식을 미분해서 구해보면  
    output layer의 dZ는 *__output - target__* 으로 간단하게 축약됩니다.  
    따라서 forward pass에서 전달받은 output과 target을 사용해 dZ를 구하고 반환합니다.

<br>

6) __calc_hidden_layer_dZ()__  
  
    Backward Pass 과정에서 hidden layer(ReLU)에서의 dZ를 계산하는 함수입니다.  
    ReLU를 사용한 dZ는, ReLU의 정의에 따라 Z값이 0보다 작거나 같으면 0, 0보다 크면 1입니다.  
    이 방식을 사용해 dZ를 구하고 반환합니다.  

<br>

7) __calc_dW()__  
  
    Backward Pass 과정에서 현재 Layer의 dW(Loss를 W로 편미분한 것)를 계산하는 함수입니다.  
    dW를 Chain Rule로 미분해보면, 계산하는데 Forward Pass에서  
    저장한, 해당 Layer의 Input(이전 Layer의 Activation)이 필요하게 됩니다.  
    따라서 이 값을 cache에서부터 불러오고, 이 값을 활용해 dW를 계산합니다.  
      
    실제 수학적 수식으로 구현하기 위해서는 Transpose를 적절하게 사용해야 하지만,  
    List를 이용해 Transpose를 구현하는 것은 메모리 관점에서 비효율적이라고 생각했습니다.  

    (예컨대, list[10][1]에서 10개의 값을 탐색하는 것은,  
    list[1][10]에서 10개의 값을 탐색하는 것이 더 비효율적이다.)  
     
    따라서 for loop에서 적절하게 식을 설정하여 동일한 연산이 수행되도록 만들었습니다.

<br>

8) __calc_db()__  

    Backward Pass 과정에서 현재 Layer의 db를 계산하는 함수입니다.

<br>

9) __calc_dA_prev()__  

    Backward Pass 과정에서, 현재 Layer로부터, 이전 Layer의 Activation(Prev Activation)에 대한  
    gradient를 구하는 함수입니다. 이것을 구하고 이전 Layer로 넘겨주어야만  
    이전 Layer의 gradient를 구할 수 있기 때문에 사용합니다.

<br>

10) __calc_softmax_gradient()__  

    Softmax를 사용하는 Layer에서의 gradient를 구하는 함수입니다.  
    dW, db를 구하기 위해서 calc_output_layer_dZ(), calc_dW(), calc_db()를 사용하고,  
    calc_dA_prev()를 구해서 이전 Layer로 넘겨줍니다.  

<br>

11) __calc_relu_gradient()__  

    ReLU를 사용하는 Layer에서의 gradient를 구하는 함수입니다.  
    dW, db를 구하기 위해서 calc_hidden_layer_dZ(), calc_dW(), calc_db()를 사용하고,  
    calc_dA_prev()를 구해서 이전 Layer로 넘겨줍니다.  

<br>

12) __backward_pass()__  

    Backward Pass의 전체 과정을 진행하는 함수입니다.  
    Backward Pass는 하나의 Example에 대해서 진행하며, 전체 Layer에서의  dW, db를 구하고,  
    이 값을 저장하는 역할을 합니다. 이후에 parameter update 함수들을 이용하여  
    실제로 parameter를 update합니다.  

<br>

13) __isCorrect()__  

    Forward Pass에서 최종적으로 나온 Output이 정답과 일치하는지 검사하는 함수입니다.  
    정답과 일치한다면 True를, 틀렸다면 False를 return하여 현재 epoch에서의 Accuracy를  
    구하는데 사용합니다.

<br>

14) __train()__  

    Training Data에 대해서 Forward Pass와 Backward Pass,  
    그리고 Parameter Update를 수행하는 함수입니다.

<br>

15) __predict()__  

    Test Data에 대해서 Forward Pass를 진행하고, 학습 결과를 Accuracy로 확인하는 함수입니다.

<br>


In [9]:
train_data_path = "./train/"
test_data_path = "./test/"

# 모델 설정

모델의 아키텍처를 설정하는 부분입니다.  

    * layer_num = 4
    * epoch_num = 200
    * learning_rate = 0.03

    * input_node_num = 256 (16*16 이미지의 flatten)
    * output_node_num = 7 (ex. [0 1 0 0 0 0 0])
    * hidden_node_num = [96, 48] (hidden layer를 구성하는 node 개수)
    
  
이 부분을 수정하여 Model의 구조를 변경하며 Train/Test를 진행할 수 있습니다.

In [10]:
# input layer를 포함한 전체 layer의 수
layer_num = 4

# epoch 횟수
epoch_num = 200

# learning_rate, 학습률
learning_rate = 0.03

input_node_num = 256 # input layer의 node num
output_node_num = 7 # output layer의 node num
hidden_node_num = [96, 48] # hidden layer들의 node num

In [11]:
random.seed(42)

* random.seed(42)

    random 함수에 대한 seed 설정

In [12]:
# Model 객체 생성
model = Model(
    layer_num=layer_num, 
    epoch_num=epoch_num, 
    learning_rate=learning_rate)

* 모델 객체를 생성한다.

In [13]:
# Model에 layer_num만큼 Layer를 추가한다.
# 이 때, input layer는 실제 Layer 객체로 구성하지는 않고
# 계산을 편리하게 하기 위해 self.layer_num list에 input_node_num만 추가한다.

model.layer_node_num.clear()
model.layers.clear()

for layer_idx in range(model.layer_num):
    # input layer
    if (layer_idx == 0):
        model.layer_node_num.append(input_node_num)
        model.insert_new_layer(activation_type='none', layer_idx=layer_idx)
        
    # output layer(activation은 softmax)
    elif (layer_idx == layer_num-1):
        model.layer_node_num.append(output_node_num)
        model.insert_new_layer(activation_type='softmax', layer_idx=layer_idx)

    # hidden layer(activation은 relu)
    else:
        model.layer_node_num.append(hidden_node_num[layer_idx-1]) # hidden node의 개수 list를 탐색하며 추가
        model.insert_new_layer(activation_type='relu', layer_idx=layer_idx)

* 모델에 Layer를 추가한다. 설정한 모델 아키텍처를 바탕으로, insert_new_layer() 메서드를 실행한다.

In [14]:
# 모델에 데이터 셋 로드
model.load_data(train_data_path, test_data_path)

* Data Set(Train_Data, Train_Label/Test_Data, Test_Label)를 Load한다.

In [15]:
model.train()

Epoch 0


Accuracy : 0.14285714285714285
Epoch 1
Accuracy : 0.11666666666666667
Epoch 2
Accuracy : 0.14047619047619048
Epoch 3
Accuracy : 0.16904761904761906
Epoch 4
Accuracy : 0.1976190476190476
Epoch 5
Accuracy : 0.2357142857142857
Epoch 6
Accuracy : 0.2761904761904762
Epoch 7
Accuracy : 0.3238095238095238
Epoch 8
Accuracy : 0.38095238095238093
Epoch 9
Accuracy : 0.45
Epoch 10
Accuracy : 0.4976190476190476
Epoch 11
Accuracy : 0.5428571428571428
Epoch 12
Accuracy : 0.6047619047619047
Epoch 13
Accuracy : 0.6476190476190476
Epoch 14
Accuracy : 0.6880952380952381
Epoch 15
Accuracy : 0.7119047619047619
Epoch 16
Accuracy : 0.7285714285714285
Epoch 17
Accuracy : 0.7357142857142858
Epoch 18
Accuracy : 0.7547619047619047
Epoch 19
Accuracy : 0.7619047619047619
Epoch 20
Accuracy : 0.7785714285714286
Epoch 21
Accuracy : 0.7928571428571428
Epoch 22
Accuracy : 0.8023809523809524
Epoch 23
Accuracy : 0.8166666666666667
Epoch 24
Accuracy : 0.8238095238095238
Epoch 25
Accuracy : 0.830952380952381
Epoch 26
Accur

* The above output is the Training Result.

In [16]:
model.predict()

Correct : 129
Total : 140
정답률 : 0.9214285714285714


* The above output is the Test Result.

# Summary

현재 설정한 Model의 Architecture는 다음과 같다.  
* Layer Num : 4  
* Layer Node Num : [256, 96, 48, 7]  
* Learning Rate : 0.03  
* Epoch Num : 200(Python)  
<br>
    
1) train() 함수를 실행하여 Training Data Set에 대해서 Forward Pass/Backward Pass/Parameter Update를 진행한다.  
    먼저 한 개의 Example에 대해서 Forward Pass/Backward Pass를 진행하여 dW, db를 구하고,  
    모든 Exaple에 대한 Gradient에 대한 평균을 구한 뒤, 이를 바탕으로 Parameter Update를 진행하여 1 Epoch을 수행한다.  
    그리고 이 과정을 지정한 Epoch 수만큼 반복하여 Training을 진행한다.  
    
2) predict() 함수를 실행하여 학습된 parameter 값을 바탕으로 140개의 Test Data Set에 대해 Forward Pass를 진행, Accuracy를 도출한다.



다양한 Hyper Parameter(Learning Rate, Layer Num, Layer Node Num, Parameter Initialization)를 바꿔가면서 실험을 진행했습니다.  
그런데, Test Data Set을 만들 때 Training Data Set에 비해서 다소 변칙적인 이미지를 많이 생성하였더니,  
해당 데이터들에 대해서는 학습 과정에서 완전히 학습되지 못해, 100%에 가까운 Accuracy를 보이지는 못했습니다.  
  
이 문제 또한 해결하기 위해 Data Augmentation/L2 Regularization과 같은 다양한 Regularization 방법을 적용해 보려고 하였으나,  
이 부분은 구현 상의 어려움과 시간 부족으로 완벽하게 실행해 보지는 못했습니다.  
  
이에 따라 일반적으로, 다음과 같은 결과를 얻게 되었습니다.  
  
  * Python: Training Data (Accuracy 약 97%), Test Data (Accuracy 약 92%) 

이후에 좀 더 다양한 방법을 적용해보면서 모델을 개선시킬 수 있는 방법을 찾아보고자 합니다.  